*COPY* FOLDER 📂

In [ ]:
!cp -r /content/drive/MyDrive/yolov5 /content/

*UNZIP* FILE 📂 ⬅️ 🤐

In [ ]:
!unzip -q ../content/yolov5.zip -d ../content/yolov5/

ZIP FOLDER 📂 ➡️ 🤐

In [ ]:
import zipfile
import os

def zip_folder(folder_path, zip_path):
    """
    Compresses the contents of a folder into a zip file.

    Args:
        folder_path (str): Path to the folder to be zipped.
        zip_path (str): Path to the resulting zip file.
    """
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, folder_path))

# Example usage:
folder_to_zip = '../yolov5'
zip_file_path = '../yolov5.zip'

zip_folder(folder_to_zip, zip_file_path)

DELETE DIRECTORY ❌

In [ ]:
import shutil

directory_to_delete = "/content/runs"

try:
    shutil.rmtree(directory_to_delete)
    print(f"Directory '{directory_to_delete}' has been successfully deleted.")
except OSError as e:
    print(f"Error: {directory_to_delete} : {e.strerror}")


Directory '/content/runs' has been successfully deleted.


MOUNT DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


SETUP

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.23 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.7/107.7 GB disk)


*LOCAL USAGE VERSION*

GET ROOT DIRECTORY

In [ ]:
ROOT_DIR = '/content/gdrive/MyDrive/yolov8'

TRAIN

In [ ]:
!pip install ultralytics

In [ ]:
import os
from ultralytics import YOLO
model = YOLO("yolov8m.pt")
results = model.train(data=os.path.join(ROOT_DIR, "custom_data.yaml"), epochs=100, batch=2)

In [ ]:
from ultralytics import YOLO

# Load a model
# model = YOLO('yolov8n.yaml')  # build a new model from scratch
model = YOLO('/content/runs/detect/train/weights/best.pt')  # load a pretrained model (recommended for training)

# Use the model
# results = model.train(data='coco128.yaml', epochs=3)  # train the model
# results = model.val()  # evaluate model performance on the validation set
results = model('/content/gdrive/MyDrive/yolov8/custom_data/images/val/v14_frame_0008.jpg')  # predict on an image
# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    print("My boxes :")
    print(boxes)
    print("Box end")

    # Extracting data tensors
    xywh_tensor = boxes.xywh
    xyxy_tensor = boxes.xyxy

    # Getting values as numpy arrays
    xywh = xywh_tensor.numpy()
    xyxy = xyxy_tensor.numpy()

    # Extracting top-left xy-coordinate, width, and height
    x_min, y_min, width, height = xywh[0][0], xywh[0][1], xywh[0][2], xywh[0][3]

    print("Top-left xy-coordinate:", (x_min, y_min))
    print("Width:", width)
    print("Height:", height)

# results = model.export(format='onnx')  # export the model to ONNX format

In [ ]:
import os

from ultralytics import YOLO
import cv2


# VIDEOS_DIR = os.path.join('.', 'custom_data', 'images', 'val') #address of video

VIDEOS_DIR = os.path.join('content', 'gdrive', 'MyDrive', 'DETECTION', 'yolov5', 'videos') #address of video

# /content/gdrive/MyDrive/DETECTION/yolov5/videos/video10.mp4

video_path = os.path.join(VIDEOS_DIR, 'video10.mp4')
video_path_out = '{}_out.mp4'.format(video_path)

video_path = 0 # for webcam

cap = cv2.VideoCapture(video_path)

ret, frame = cap.read()
H, W, _ = frame.shape

if video_path != 0:
  out = cv2.VideoWriter(video_path_out, cv2.VideoWriter_fourcc(*'MP4V'), int(cap.get(cv2.CAP_PROP_FPS)), (W, H))

model_path = os.path.join('.', 'runs', 'detect', 'train', 'weights', 'best.pt')

# Load a model
model = YOLO(model_path)  # load a custom model

threshold = 0.75

while ret:

    results = model(frame)[0]

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        if score > threshold:
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
            cv2.putText(frame, results.names[int(class_id)].upper(), (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)

    if video_path != 0:
      out.write(frame)
    cv2.imshow(frame, "output")
    ret, frame = cap.read()

cap.release()
if video_path != 0:
  out.release()
cv2.destroyAllWindows()

In [ ]:
!scp -r /content/runs '/content/gdrive/MyDrive/yolov8'
# !scp -r '/content/gdrive/MyDrive/yolov8/runs' /content/runs

In [ ]:
!scp /content/yolov8n.pt '/content/gdrive/MyDrive/yolov8'
# !scp '/content/gdrive/MyDrive/yolov8/yolov8m.pt' /content/

TRAIN

SETUP LOGGER

In [ ]:
#@title Select YOLOv8 🚀 logger {run: 'auto'}
logger = 'Comet' #@param ['Comet', 'TensorBoard']

if logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.init()
elif logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir .

TRAINING CODE

In [ ]:
!yolo train model=yolov8m.pt data=/content/gdrive/MyDrive/yolov8/custom_data.yaml epochs=100 imgsz=640

VALIDATE

In [ ]:
!yolo val model=/content/runs/detect/train/weights/best.pt data=/content/gdrive/MyDrive/yolov8/custom_data.yaml

TEST

In [ ]:
# Run inference on an image with YOLOv8n
!yolo predict model=/content/runs/detect/train/weights/best.pt source='/content/gdrive/MyDrive/yolov5/videos/video10.mp4'

EXPORT

Export a YOLOv8 model to any supported format below with the `format` argument, i.e. `format=onnx`. See [YOLOv8 Export Docs](https://docs.ultralytics.com/modes/export/) for more information.

- 💡 ProTip: Export to [ONNX](https://onnx.ai/) or [OpenVINO](https://docs.openvino.ai/latest/index.html) for up to 3x CPU speedup.  
- 💡 ProTip: Export to [TensorRT](https://developer.nvidia.com/tensorrt) for up to 5x GPU speedup.


| Format                                                             | `format` Argument | Model                     | Metadata | Arguments                                           |
|--------------------------------------------------------------------|-------------------|---------------------------|----------|-----------------------------------------------------|
| [PyTorch](https://pytorch.org/)                                    | -                 | `yolov8n.pt`              | ✅        | -                                                   |
| [TorchScript](https://pytorch.org/docs/stable/jit.html)            | `torchscript`     | `yolov8n.torchscript`     | ✅        | `imgsz`, `optimize`                                 |
| [ONNX](https://onnx.ai/)                                           | `onnx`            | `yolov8n.onnx`            | ✅        | `imgsz`, `half`, `dynamic`, `simplify`, `opset`     |
| [OpenVINO](https://docs.openvino.ai/latest/index.html)             | `openvino`        | `yolov8n_openvino_model/` | ✅        | `imgsz`, `half`, `int8`                             |
| [TensorRT](https://developer.nvidia.com/tensorrt)                  | `engine`          | `yolov8n.engine`          | ✅        | `imgsz`, `half`, `dynamic`, `simplify`, `workspace` |
| [CoreML](https://github.com/apple/coremltools)                     | `coreml`          | `yolov8n.mlpackage`       | ✅        | `imgsz`, `half`, `int8`, `nms`                      |
| [TF SavedModel](https://www.tensorflow.org/guide/saved_model)      | `saved_model`     | `yolov8n_saved_model/`    | ✅        | `imgsz`, `keras`, `int8`                            |
| [TF GraphDef](https://www.tensorflow.org/api_docs/python/tf/Graph) | `pb`              | `yolov8n.pb`              | ❌        | `imgsz`                                             |
| [TF Lite](https://www.tensorflow.org/lite)                         | `tflite`          | `yolov8n.tflite`          | ✅        | `imgsz`, `half`, `int8`                             |
| [TF Edge TPU](https://coral.ai/docs/edgetpu/models-intro/)         | `edgetpu`         | `yolov8n_edgetpu.tflite`  | ✅        | `imgsz`                                             |
| [TF.js](https://www.tensorflow.org/js)                             | `tfjs`            | `yolov8n_web_model/`      | ✅        | `imgsz`                                             |
| [PaddlePaddle](https://github.com/PaddlePaddle)                    | `paddle`          | `yolov8n_paddle_model/`   | ✅        | `imgsz`                                             |
| [NCNN](https://github.com/Tencent/ncnn)                            | `ncnn`            | `yolov8n_ncnn_model/`     | ✅        | `imgsz`, `half`                                     |


In [ ]:
!yolo export model=yolov8n.pt format=torchscript

PYTHON USAGE

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.yaml')  # build a new model from scratch
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

# Use the model
results = model.train(data='coco128.yaml', epochs=3)  # train the model
results = model.val()  # evaluate model performance on the validation set
results = model('https://ultralytics.com/images/bus.jpg')  # predict on an image
results = model.export(format='onnx')  # export the model to ONNX format